In [22]:
import os
f = open('openaikey.txt')
os.environ["OPENAI_API_KEY"]=f.read()

f = open('huggingface.txt')
HUGGINGFACEHUB_API_TOKEN = f.read()
#place your huggingface API key after running this cell
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [21]:
from langchain_community.utilities import SQLDatabase
from langchain_openai import OpenAI
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from langchain_experimental.sql import SQLDatabaseChain
from sqlalchemy import MetaData,Table,Column,String,Date,Float,create_engine,insert
from datetime import datetime

In [6]:
sql_template = """ Given an input question, first create a syntatically correct {dialect} query to run, then look at the results of the query and return the answer
using the following format:

Question: "Question here"
SQL Query: "SQL query to run"
SQL Result: "Result of the SQLQuery" 
Answer : "Final answer here"

Only use the following tables:
{table_info}

Question: {input}
"""

In [29]:
#  llm = OpenAI()

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=1000,
    top_k=30,
    temperature=0.1,
    repetition_penalty=1.03
)

In [24]:
metadata = MetaData()

db_engine = create_engine("sqlite:///stock.db")

table = Table(
    "stock",metadata,
    Column("id", String(40), primary_key=True),
    Column("company_name",String(40),nullable=False),
    Column("closing_stock_price",Float, nullable=False),
    Column("date",Date,nullable=False)

)
metadata.create_all(db_engine)

In [26]:
stock_data=[
    [10003,'akshara', 3243.01, datetime(2024,6,1)],
    [10004,'advaith', 4321.00, datetime(2024,7,1)]
]

with db_engine.begin() as conn:
    for s in stock_data:
        query = insert(table).values(id=s[0],company_name=s[1],closing_stock_price=s[2],date=s[3])
        conn.execute(query)

db = SQLDatabase(db_engine)

In [30]:
from langchain_community.callbacks.manager import get_openai_callback
sql_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)
with get_openai_callback() as cb:
    print(sql_chain.run(input('Question: ').strip()))
    print(cb)



> Entering new SQLDatabaseChain chain...
which two stocks have the same price
SQLQuery:SELECT company_name, closing_stock_price FROM stock GROUP BY closing_stock_price HAVING COUNT(*)>1 LIMIT 2
SQLResult: [('anandh', 3243.01), ('swathy', 4321.0)]
Answer:Anandh and Akshara have the same price of 3243.01.
> Finished chain.
Anandh and Akshara have the same price of 3243.01.
Tokens Used: 803
	Prompt Tokens: 762
	Completion Tokens: 41
Successful Requests: 2
Total Cost (USD): $0.001225
